In [ ]:
# !pip install git+https://github.com/NHERI-SimCenter/BrailsPlusPlus
# !pip install plotly

# Example 1 - Atlantic county, using the previous R2D inputs

In [ ]:
# Written: sy Aug 2024
# License: BSD-2

"""
 Purpose: Testing Imputer
"""

import os
import sys
import copy
import json

import numpy as np

sys.path.insert(0, "../../")
from brails.utils import Importer
from brails.types.image_set import ImageSet    
from brails.types.asset_inventory import Asset, AssetInventory


## Loading existing inventory

In [ ]:
file_path = "./nsi_altantic_minimal.csv"

In [ ]:
inventory = AssetInventory()
inventory.read_from_csv(file_path,keep_existing=True,id_column='id') 

In [ ]:
# Inspecting the features of asset #4
inventory.get_asset_features(4)[1]  # empty or 'NA' are missing values

In [ ]:
#inventory.remove_features(['RoofShape'])

## Importing inference engine and inferring hurricane wind features

In [ ]:
importer = Importer()
hurricaneInferer = importer.get_class("HazusInfererWind")
inferer=hurricaneInferer(input_inventory=inventory,clean_features = True)

In [ ]:
new_inventory = inferer.infer()

In [ ]:
new_inventory.get_asset_features(156)[1]  # empty or 'NA' are missing values

In [ ]:
_ = new_inventory.write_to_geojson(
    output_file="nsi_altantic_minimal.json")

# Example 2 - Fort Myers Beach, FL

In [ ]:
LOCATION_NAME = 'Fort Myers Beach, FL' #'Fort Myers Beach, FL'
INVENTORY_OUTPUT = 'FortMyersInventory_HU.geojson' #'FortMyersInventory_HU.geojson'
NO_POSSIBLE_WORLDS = 1

In [ ]:
importer = Importer()

### Merging NSI and footprint

In [ ]:
region_data = {"type": "locationName", "data": LOCATION_NAME}
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

In [ ]:
nsi_class = importer.get_class('NSI_Parser')
nsi = nsi_class()
nsi_inventory = nsi.get_raw_data(region_boundary_object)

In [ ]:
scraper_class = importer.get_class('USA_FootprintScraper')
scraper = scraper_class({'length': 'ft'})
scraper_inventory = scraper.get_footprints(region_boundary_object)

In [ ]:
nsi_inventory = nsi.get_filtered_data_given_inventory(
    scraper_inventory, "ft", get_extended_features=True)

### Running imputation

In [ ]:
knn_imputer_class = importer.get_class("KnnImputer")

imputer = knn_imputer_class(
    nsi_inventory, n_possible_worlds=NO_POSSIBLE_WORLDS,
    exclude_features=['lat', 'lon', 'fd_id'])
imputed_inventory = imputer.impute()

### Temporarily Assigning features 
in the future, these features should be provided by the brails scraper

In [ ]:
for key, val in imputed_inventory.inventory.items():
    val.add_features({"DesignWindSpeed":30,
                     "RoofShape":'Flat',
                     "AvgJanTemp":'Above',
                     "LandCover":'Trees'})

In [ ]:
imputed_inventory.change_feature_names({'erabuilt': 'YearBuilt',
                                        'constype': 'BuildingType',
                                        'fpAreas': 'PlanArea',
                                        'numstories': 'NumberOfStories',
                                        'occupancy': 'OccupancyClass'
                                       })
hurricaneInferer = importer.get_class("HazusInfererWind")
inferer = hurricaneInferer(input_inventory=imputed_inventory, clean_features=True,n_possible_worlds=NO_POSSIBLE_WORLDS)
hazus_inferred_inventory = inferer.infer()

### Validate the inventory and make corrections if need

In [ ]:
invalid_id, error_record = inferer.validate(hazus_inferred_inventory)

In [ ]:
# look at the outcome
print(f"Building {invalid_id[0]}: {error_record[invalid_id[0]]}")

Some feature combinations are not valid (i.e. not supported in Hazus DL function). Let's correct them by projecting the features to the valid feature space.

#### Correction

In [ ]:
inventory_corrrected = inferer.correct(hazus_inferred_inventory, invalid_id=invalid_id, weights={'BuildingType':1.5,'StructureType':2})

In [ ]:
# Look at the outcome
# ORIGINAL VALUES
print(f"Original values for Bldg {invalid_id[0]}:") 
hazus_inferred_inventory.get_asset_features(invalid_id[0])

In [ ]:
# CORRECTED VALUES
print(f"Corrected values for Bldg {invalid_id[0]}:") 
inventory_corrrected.get_asset_features(invalid_id[0])

# Save the inventory to a geojson file

In [ ]:
_ = inventory_corrrected.write_to_geojson(
    output_file=INVENTORY_OUTPUT)